In [3]:
%matplotlib inline

import os
import pickle as pkl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from data.stanford_dogs import StanfordDogs

BATCH_SIZE = 32

In [4]:
"""
Yarne Hermann YPH2105
"""

train_dataset = StanfordDogs('./images')
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)



In [ ]:
"""
implementation of original discriminator
"""
"""
J.R. Carneiro JC4896
"""

class Discriminator(nn.Module):
    
    def __init__(self, num_classes=1):
        super(Discriminator, self).__init__()
        
        v=256*256*3
        n=int(((v-4*4+2)/2+1)*32) #((input_volume−kernel_volume+2padding)/stride+1)*numberOfLayers
        
        self.conv1 = self.conv(v, n, 4, 2,1) #(32 4x4)
        self.conv2 = self.conv(n,2*n) #(64 4x4)
        self.conv3 = self.conv(2*n, 4*n) #(128 4x4)
        self.conv4 = self.conv(4*n, 8*n) #(256 4x4)
        self.conv5 = self.conv(8*n, 16*n) #(512 4x4)
        self.conv6 = self.conv(16*n, 16*n) #(512 4x4)
        
        self.multi1 = nn.Linear(16*n, 1024)
        self.multi2 = nn.Linear(1024, 512)
        self.multi3 = nn.Linear(512, num_classes)
        
        self.real = nn.Linear(16*n, 2)

    #helper method to create convolutional layers
    def conv(self, in_channels, out_channels, kernel_size=4, stride=2, padding=1):
        return nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
    
    def forward(self, x):
        out = F.leaky_relu((self.conv1(x)), 0.1)
        out = F.leaky_relu((self.conv2(out)), 0.1)
        out = F.leaky_relu((self.conv3(out)), 0.1)
        out = F.leaky_relu((self.conv4(out)), 0.1)
        out = F.leaky_relu((self.conv5(out)), 0.1)
        out = F.leaky_relu((self.conv6(out)), 0.1)
        
        out.flatten()
        
        multi_output = self.multi1(out)
        multi_output = self.multi2(multi_output)
        multi_output = self.multi3(multi_output)
        multi_output = F.softmax(multi_output)
        
        real_output = self.real(out)
        real_output = self.sigmoid(real_output)
        
        return real_output, multi_output


In [ ]:
"""
implementation of original generator
"""
"""
J.R. Carneiro JC4896
"""

class Generator(nn.Module):
    
    def __init__(self):
        super(Generator, self).__init__()
        conv_dim = 256*256*3
        
        self.fc = nn.Linear(100, conv_dim*2*2*2*2*2*2*2)
        
        self.t_conv1 = self.deconv(conv_dim*2*2*2*2*2*2*2, conv_dim*2*2*2*2*2*2)
        self.t_conv2 = self.deconv(conv_dim*2*2*2*2*2*2, conv_dim*2*2*2*2*2)
        self.t_conv3 = self.deconv(conv_dim*2*2*2*2*2, conv_dim*2*2*2*2)
        self.t_conv4 = self.deconv(conv_dim*2*2*2*2, conv_dim*2*2*2)
        self.t_conv5 = self.deconv(conv_dim*2*2*2, conv_dim*2*2)
        self.t_conv6 = self.deconv(conv_dim*2*2, conv_dim*2)
        self.t_conv7 = self.deconv(conv_dim*2, conv_dim)
        
    #helper method to create deconvolutional layers
    def deconv(self,in_channels, out_channels, kernel_size=4, stride=2, padding=1):
        return nn.ConvTranspose2d(in_channels, out_channels,kernel_size,stride,padding, bias=False)
    
    def forward(self, x):
        out = self.fc(x)
        out = out.view(-1, 2048) 
        
        out = F.leaky_relu(t_conv1(out),0.1)
        out = F.leaky_relu(t_conv2(out),0.1)
        out = F.leaky_relu(t_conv3(out),0.1)
        out = F.leaky_relu(t_conv4(out),0.1)
        out = F.leaky_relu(t_conv5(out),0.1)
        out = F.leaky_relu(t_conv6(out),0.1)
        out = F.leaky_relu(t_conv7(out),0.1)
        
        out = F.tanh(out)
        
        return out

In [ ]:
D = Discriminator()

In [ ]:
G = Generator()

In [ ]:
""" 
FROM Udacity DCGAN implementation
"""
def real_loss(D_out, smooth=False):
    # batch_size = D_out.size(0)
    # label smoothing
    if smooth:
        # smooth, real labels = 0.9
        labels = torch.ones(BATCH_SIZE)*0.9
    else:
        labels = torch.ones(BATCH_SIZE) # real labels = 1
    # move labels to GPU if available     
    if train_on_gpu:
        labels = labels.cuda()
    # binary cross entropy with logits loss
    criterion = nn.BCEWithLogitsLoss()
    # calculate loss
    loss = criterion(D_out.squeeze(), labels)
    return loss

In [ ]:
"""
MODIFIED Udacity DCGAN implementation
"""
"""
J.R. Carneiro JC4896
"""
def multi_loss(D_out, labels):
    # batch_size = D_out.size(0)
    # labels = torch.zeros(batch_size) # fake labels = 0
    if train_on_gpu:
        labels = labels.cuda()
    criterion = nn.CrossEntropyLoss() 
    loss = criterion(D_out.squeeze(), labels)
    return loss

In [6]:
"""
Yarne Hermann YPH2105
"""
# Have to make sure to be correct about maximizing or minimizing loss.
# I took the negative of what is mentioned on page 9 in the paper in order to create a loss
# to be minimized. If I'm correct real_loss can be used as it is right now
def entropy_loss(D_out):
    ### TO BE COMPLETED
    K = train_dataset.NUM_CLASSES
    loss = torch.zeros(BATCH_SIZE)
    
    # softmaxing
    # e = torch.exp(D_out)
    # s = torch.sum(e, dim=1)
    # probabilities = e / s.view(BATCH_SIZE, 1)
    
    # Just regular normalization
    probabilities = D_out / torch.sum(D_out, dim=1).view(BATCH_SIZE, 1)
    
    print(probabilities)
            
    for c in range(K):
        # labels = torch.ones(batch_size) * c
        # if train_on_gpu:
        #     labels = labels.cuda()
        
        c_loss = - (1/K * torch.log(probabilities[:, c]) + (1 - 1/K) * torch.log(torch.ones(BATCH_SIZE)-probabilities[:, c]))         
        loss += c_loss
    return loss
        
'''
test
''' 
D_out_min_entropy = torch.zeros(BATCH_SIZE, train_dataset.NUM_CLASSES)
for i in range(BATCH_SIZE):
    D_out_min_entropy[i][0] = 1
D_out_random = torch.rand(BATCH_SIZE, train_dataset.NUM_CLASSES)

D_out_max_entropy = torch.ones(BATCH_SIZE, train_dataset.NUM_CLASSES) 

print(entropy_loss(D_out_min_entropy))
print(entropy_loss(D_out_random))
print(entropy_loss(D_out_max_entropy))


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])
tensor([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf])
tensor([[0.0057, 0.0159, 0.0150,  ..., 0.0003, 0.0152, 0.0035],
        [0.0125, 0.0026, 0.0011,  ..., 0.0057, 0.0005, 0.0067],
        [0.0093, 0.0155, 0.0134,  ..., 0.0081, 0.0097, 0.0117],
        ...,
        [0.0142, 0.0067, 0.0149,  ..., 0.0019, 0.0154, 0.0153],
        [0.0085, 0.0061, 0.0090,  ..., 0.0034, 0.0124, 0.0022],
        [0.0013, 0.0118, 0.0033,  ..., 0.0084, 0.0045, 0.0152]])
tensor([6.0636, 6.0934, 6.1159, 5.9970, 6.0932, 6.1287, 6.0779, 6.1407, 6.0515,
        6.0578, 6.0718, 6.0623, 6.1187, 6.1434, 6.1215, 6.1283, 6.2140, 6.0174,
        6.1

In [ ]:
""" 
FROM Udacity DCGAN implementation
"""
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    G.cuda()
    D.cuda()
    print('GPU available for training. Models moved to GPU')
else:
    print('Training on CPU.')

In [ ]:
"""
MODIFIED Udacity DCGAN implementation
"""
"""
J.R. Carneiro JC4896
"""
num_epochs = 50

# keep track of loss and generated, "fake" samples
samples = []
losses = []
print_every = 300
sample_size=16

In [ ]:
"""
J.R. Carneiro JC4896
"""
# to collect samples from the generator
fixed_z = torch.from_numpy(np.random.uniform(-1, 1, size=(sample_size, 100))).float()

# 2.
for epoch in range(num_epochs):
    
    for batch_i, (real_images, real_labels) in enumerate(train_dataloader):
                
        # batch_size = real_images.size(0)
        
        # important rescaling step
        #real_images = scale(real_images)
        
        g_optimizer.zero_grad()
        
        # 3.
        z = np.random.uniform(-1, 1, size=(BATCH_SIZE, z_size)) 
        z = torch.from_numpy(z).float()
        if train_on_gpu:
            z = z.cuda()
            
        # 4.
        fake_images = G(z)
        d_optimizer.zero_grad()
        
        if train_on_gpu:
            real_images = real_images.cuda()
        
        # 5.
        D_real, D_multi = D(real_images)
        d_real_real_loss = real_loss(D_real) 
        # 6.
        d_real_multi_loss = multi_loss(D_multi, real_labels)
        # 7.
        D_fake = D(fake_images)
        d_fake_real_loss = real_loss(D_fake)
        # 8.
        g_fake_entropy_loss = entropy_loss(D_fake) ##
        
        # 9.
        d_loss= log(d_real_real_loss)+log(d_real_multi_loss)+log(1-d_fake_real_loss)
        
        # 10.
        d_loss.backward()
        d_optimizer.step()
        
        # 11.
        g_loss=log(d_fake_real_loss)-g_fake_entropy_loss
        
        # 12.
        g_loss.backward()
        g_optimizer.step()
        
        
### START - FROM Udacity DCGAN implementation ###
        # Print some loss stats
        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            losses.append((d_loss.item(), g_loss.item()))
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, d_loss.item(), g_loss.item()))

    
    ## AFTER EACH EPOCH##    
    # generate and save sample, fake images
    G.eval() # for generating samples
    if train_on_gpu:
        fixed_z = fixed_z.cuda()
    samples_z = G(fixed_z)
    samples.append(samples_z)
    G.train() # back to training mode

# Save training generator samples
with open('train_samples.pkl', 'wb') as f:
    pkl.dump(samples, f)
    
### END -   FROM Udacity DCGAN implementation ###